In [1]:
import sys, os
from os import path
# sys.path.append('/root/bdrisk/risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

In [40]:
org_code = '91310000132612172J'

In [55]:
p0=f'''select MODEL_CODE,SCORE from {TableList.BD_RISK_CORP_SCORE_DISPLAY.value} WHERE ISCURRENT = 1 AND CUSTOMER_CODE = 'FTA_LG' AND MODEL_CODE IN ('CREDIT','FINANCE','PRODUCTION','SUPPLYCHAIN','STOCK','TRACK','PUBLIC','CUSTOM') AND ORG_CODE LIKE '{org_code}'  '''
score_list = Read_Oracle().read_oracle(sql= p0, database = 'dbdm')

model_list = ['CREDIT','FINANCE','PRODUCTION','SUPPLYCHAIN','STOCK','TRACK','PUBLIC','CUSTOM']
df_model = pd.DataFrame(model_list,columns=['MODEL_CODE'])

score_result = pd.merge(df_model,score_list,how='left',on=['MODEL_CODE'])
score_result['SCORE'].fillna(100,inplace=True)

total_score = round(score_result['SCORE'].mean(),2)

2021-12-08 16:21:47.549 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 0.230s


In [63]:
model_list = ['CREDIT','FINANCE','PRODUCTION','SUPPLYCHAIN','STOCK','TRACK','PUBLIC','CUSTOM']
df_model = pd.DataFrame(model_list,columns=['MODEL_CODE'])

In [64]:
score_result = pd.merge(df_model,score_list,how='left',on=['MODEL_CODE'])

In [69]:
score_result['SCORE'].fillna(100,inplace=True)

In [73]:
total_score = round(score_result['SCORE'].mean(),2)

In [ ]:
p1=f'''select SCORE from {TableList.BD_RISK_CORP_SCORE_DISPLAY.value} WHERE ISCURRENT = 1 AND CUSTOMER_CODE = 'FTA_LG' AND MODEL_CODE = 'TOTAL' AND ORG_CODE LIKE '{org_code}' ORDER BY LASTUPDATE DESC  '''
pre_total = Read_Oracle().read_oracle(sql= p1, database = 'dbdm')

if pre_total is not None:
    pre_sc = pre_total[0,'SCORE']
    score_change = total_score - pre_sc
    CHANGE_RESULT = pd.DataFrame(data=[[0, self.org_code, 'CHANGE', score_change]], columns=['ID', 'ORG_CODE', 'MODEL_CODE', 'SCORE'])
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    CHANGE_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
    CHANGE_RESULT['CUSTOMER_CODE'] = 'FTA_LG'
    Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',CHANGE_RESULT, org_code = self.org_code, alarm = ['CHANGE',''])
else:
    print(f'Notification: pre total score not found. no score change written. org_code:{org_code}')

TOTAL_RESULT = pd.DataFrame(data=[[0, self.org_code, 'TOTAL', total_score]], columns=['ID', 'ORG_CODE', 'MODEL_CODE', 'SCORE'])
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
TOTAL_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
TOTAL_RESULT['CUSTOMER_CODE'] = 'FTA_LG'
Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',TOTAL_RESULT, org_code = self.org_code, alarm = ['TOTAL',''])

In [ ]:
import sys, os
from os import path
sys.path.append('/root/bdrisk/risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

class ScoreTotal(object):
    def __init__(self, child_task_id, org_code, params):
        self.child_task_id = child_task_id
        Risk_logger(child_task_id = self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code

    def score_public(self):
        #读取企业名称
        p0=f'''select ORG_CODE,ORG_NAME from {TableList.BD_RISK_CORP_INFO_BASIC.value} WHERE ORG_CODE LIKE '{self.org_code}'  '''
        corp_list = Read_Oracle().read_oracle(sql= p0, database = 'dbdm')
        if corp_list is not None:
            org_name = corp_list.loc[0,'ORG_NAME']
        else:
            print(f'Error: Corp Not Found. org_code = {self.org_code}')
            return

        # 读取企业库存模块的结果表数据
        sql=f"""\
        select RISK_LABEL from {TableList.MX_PUBLIC_ELECTRIC.value} WHERE ISCURRENT = 1 AND CORP_NAME LIKE '{org_name}' \
        """
        pb1_result=Read_Oracle().read_oracle(sql=sql,database='dbdm')

        sql=f"""\
        select RISK_LABEL from {TableList.MX_PUBLIC_INSURANCE.value} WHERE ISCURRENT = 1 AND CORP_NAME LIKE '{org_name}' \
        """
        pb2_result=Read_Oracle().read_oracle(sql=sql,database='dbdm')

        if ((pb1_result is None) and (pb2_result is None)):
            print(f'Error: No public data of this corp found. org_code = {self.org_code}, org_name = {org_name}')
            return

        if pb1_result is not None:
            sc1 = pb1_result.copy()
            sc1['SCORE'] = pb1_result['RISK_LABEL'].apply(lambda x : 0.1 if x=='比值正常' else -0.1)
            sc1_total = sc1['SCORE'].sum()
        else:
            print("Warnning: public_electric data not found.")
            sc1_total = 0

        if pb2_result is not None:
            sc2 = pb2_result.copy()
            sc2['SCORE'] = pb2_result['RISK_LABEL'].apply(lambda x : 0.1 if x=='比值正常' else -0.1)
            sc2_total = sc2['SCORE'].sum()
        else:
            print("Warnning: public_insurance data not found.")
            sc2_total = 0

        sc_total = sc1_total + sc2_total
        if sc_total>=0:
            sc_total = 100
        else:
            sc_total = 100 + sc_total

        PUBLIC_RESULT = pd.DataFrame(data=[[0, self.org_code, 'PUBLIC', sc_total]], columns=['ID', 'ORG_CODE', 'MODEL_CODE', 'SCORE'])
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        PUBLIC_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
        PUBLIC_RESULT['CUSTOMER_CODE'] = 'FTA_LG'
        Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',PUBLIC_RESULT, org_code = self.org_code, alarm = ['PUBLIC',''])

    def run_score_public(self):
        try:
            self.score_public()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id = self.child_task_id, exec_status=exec_status).write_log()

if __name__ == '__main__':
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    child_task_id = sys.argv[1]
    # child_task_id = 'fef0424300744209860f05cd2fa3daaa'
    org_code, params, base_time = read_log_table(child_task_id)
    # org_code = '91310000132612172J'
    ScorePb(child_task_id, org_code, params).run_score_public()